In [ ]:
%pip install transformers[torch] datasets accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.wh

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments


In [ ]:
model_path = "/content/drive/MyDrive/Lm/gpt2-wikitext2-model.pth"
model = GPT2LMHeadModel.from_pretrained('gpt2')
# Load the saved state dictionary on CPU
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
dataset = load_dataset("bookcorpus")


Generating train split:   0%|          | 0/74004228 [00:00<?, ? examples/s]

In [ ]:
small_dataset = dataset["train"].shuffle(seed=42).select(range(2000000))


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)  # Smaller block size


In [ ]:
tokenized_datasets = small_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Convert to PyTorch tensors
tokenized_datasets.set_format("torch")


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling

# Create DataLoader
dataloader = DataLoader(tokenized_datasets, batch_size=8, shuffle=True)

# Create Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [ ]:
# Load the pre-trained model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Define training arguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/result',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    fp16=True,  # Enable mixed precision training

)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets,
)

# Start training
trainer.train()
trainer.save_model()
tokenizer.save_pretrained(training_args.output_dir)


Step,Training Loss
100,4.438100
200,4.109900
300,4.063600
400,4.018500
500,3.982500
600,3.986800
700,3.979700
800,3.967800
900,3.976500
1000,3.938600


('/content/drive/MyDrive/result/tokenizer_config.json',
 '/content/drive/MyDrive/result/special_tokens_map.json',
 '/content/drive/MyDrive/result/vocab.json',
 '/content/drive/MyDrive/result/merges.txt',
 '/content/drive/MyDrive/result/added_tokens.json')

In [ ]:
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 74004228
    })
})


In [ ]:
import transformers
import datasets
import accelerate

print(f"Transformers version: {transformers.__version__}")
print(f"Datasets version: {datasets.__version__}")
print(f"Accelerate version: {accelerate.__version__}")


Transformers version: 4.41.0
Datasets version: 2.19.1
Accelerate version: 0.30.1


In [ ]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/results/checkpoint-5500-20240526T111640Z-001.zip"
extract_dir = "/content/drive/MyDrive/extracted_checkpoint"

# Create the directory if it doesn't exist
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Files in the extracted directory:")
print(os.listdir(extract_dir))


Files in the extracted directory:
['checkpoint-5500']


In [ ]:
model_pth = "/content/drive/MyDrive/result"


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model

def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(sequence, max_length):
    model_path = model_pth  # Use the path to the extracted directory
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)

    # Check if CUDA is available and move model to the appropriate device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    input_ids = tokenizer.encode(sequence, return_tensors='pt').to(device)
    attention_mask = tokenizer(sequence, return_tensors='pt')["attention_mask"].to(device)

    # Generate text
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=1.5,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_text)

# Example usage
sequence = input("Enter a prompt: ")  # e.g., "Once a vampire fell in love with a pixie"
max_len = int(input("Enter max length of generated text: "))  # e.g., 100
generate_text(sequence, max_len)


Enter a prompt: once upon time
Enter max length of generated text: 100
once upon time, he heard a familiar sound. lennell looked back a couple rows from her bedroom, in the far distance, to see another of their two sons kneeling in front of a large bed of white pine, a lighted wooden bed. they looked down. then they turned, and found matt, her son? snot hanging from the door behind the headstones. `` matt? '' lily raised another eyebrow. she said nothing, instead thinking in a rather unorganized and dis


In [ ]:
import pandas as pd
from datasets import load_dataset, DatasetDict


split_ratio = [0.8, 0.1, 0.1]  # 80% train, 10% validation, 10% test
train_testvalid = small_dataset.train_test_split(test_size=split_ratio[1] + split_ratio[2], shuffle=True, seed=42)
test_valid = train_testvalid["test"].train_test_split(test_size=split_ratio[2] / (split_ratio[1] + split_ratio[2]), shuffle=True, seed=42)

# Create DatasetDict
dataset_dict = DatasetDict({
    "train": train_testvalid["train"],
    "validation": test_valid["train"],
    "test": test_valid["test"]
})

# Verify the splits
print(dataset_dict)


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 80000
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 10000
    })
})


In [ ]:
import math

# Evaluate the model on the validation set
eval_results = trainer.evaluate()

# Calculate perplexity
perplexity = math.exp(eval_results["eval_loss"])
print(f"Perplexity: {perplexity}")


Perplexity: 205.38
